In [13]:
import pandas as pd
import numpy as np

import nltk
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('spanish')

from matplotlib import pyplot

import networkx as nx

from utils import description_to_words

import itertools

#### Cargamos y visualizamos el dataset

In [2]:
samples = pd.read_csv('datasets/comida_dataset.csv')
samples.head()

,section_name,category_name,category_tag_name,product_name,product_description
0,Promociones,Chivitos,Canadiense al plato,Chivito canadiense al plato para 2 con gramajo,"2 churrascos de lomo con jamón, muzza, panceta..."
1,Combinados de sushi,Sushi,sushi,Combinado Magnum - 30 piezas variadas,"Variedad de rolls, nigiris y sashimis."
2,Cucuruchos,Helados,cucurucho,Cucurucho natural,NaN
3,Sugerencia del día,Pescados y Mariscos,Merluza,Sugerencia - Merluza fresca con papas,NaN
4,Almuerzos por $99,Comida Árabe,Shawarma,Almuerzos por $99 - Shawarma Vegetariano,"Precio Anterior: $200, Ahora: $99."


In [3]:
samples.shape 

(99396, 5)

#### Rellenamos blancos con caracter nulo

In [4]:
samples["section_name"].fillna("", inplace = True)
samples["category_tag_name"].fillna("", inplace = True)
samples["product_name"].fillna("", inplace = True) 
samples["product_description"].fillna("", inplace = True)

#### Nos quedamos con los que se relacionan a nuestro producto:

In [5]:
our_target_words = '"' + '|'.join(['hamburguesa', 'Hamburguesa', 'burger', 'Burger', 'sanguche', 'Sanguche', 'sándwich', 'Sándwich']) + '|\+"'
indexNames = samples[samples["product_name"].str.contains(our_target_words)].index

In [6]:
len(indexNames)

7173

In [7]:
samples = samples.iloc[indexNames]
samples.head()

,section_name,category_name,category_tag_name,product_name,product_description
33,Hamburguesas vegetarianas,Comida Vegetariana,hamburguesa vegetariana,Hamburguesa vegetariana ciser,"Hamburguesa de garbanzos y vegetales, Lechuga,..."
66,Vegetarianas burger,Comida Vegetariana,Vegetariana,Godiee burger,"200 g de carne de lentejas al pan con rúcula, ..."
91,Hamburguesas caseras,Comida Vegetariana,hamburguesa vegetariana,Hamburguesa vegetariana,"Doble hamburguesa de lentejas y verduras,cebol..."
181,Hamburguesas,Comida Vegetariana,hamburguesa vegetariana,Hamburguesa vegetariana,"Pan artesanal con sésamo, hamburguesa de espin..."
258,Los de siempre,Comida Vegetariana,hamburguesa vegetariana,Hamburguesa Vegetariana en pan batta,"Dos hamburguesas, con mayonesa, lechuga, tomat..."


### Preprocesamiento

#### Generamos una nueva columna con la información contenido por el nombre del producto y su descripción

In [8]:
samples["description"] = samples["product_name"] + ' ' + samples["product_description"]
samples

,section_name,category_name,category_tag_name,product_name,product_description,description
33,Hamburguesas vegetarianas,Comida Vegetariana,hamburguesa vegetariana,Hamburguesa vegetariana ciser,"Hamburguesa de garbanzos y vegetales, Lechuga,...",Hamburguesa vegetariana ciser Hamburguesa de g...
66,Vegetarianas burger,Comida Vegetariana,Vegetariana,Godiee burger,"200 g de carne de lentejas al pan con rúcula, ...",Godiee burger 200 g de carne de lentejas al pa...
91,Hamburguesas caseras,Comida Vegetariana,hamburguesa vegetariana,Hamburguesa vegetariana,"Doble hamburguesa de lentejas y verduras,cebol...",Hamburguesa vegetariana Doble hamburguesa de l...
181,Hamburguesas,Comida Vegetariana,hamburguesa vegetariana,Hamburguesa vegetariana,"Pan artesanal con sésamo, hamburguesa de espin...",Hamburguesa vegetariana Pan artesanal con sésa...
258,Los de siempre,Comida Vegetariana,hamburguesa vegetariana,Hamburguesa Vegetariana en pan batta,"Dos hamburguesas, con mayonesa, lechuga, tomat...",Hamburguesa Vegetariana en pan batta Dos hambu...
...,...,...,...,...,...,...
97882,Nuestras hamburguesas caseras (300 g),Hamburguesas,Hamburguesa,Jesse Pinkman Burger,"Medallón de 300 g de carne, mayonesa, lechuga,...",Jesse Pinkman Burger Medallón de 300 g de carn...
97884,Hamburguesas,Hamburguesas,Hamburguesa,Hamburguesa doble,Doble carne casera huevo frito papá fritas jam...,Hamburguesa doble Doble carne casera huevo fri...
97885,Hamburguesas,Hamburguesas,Hamburguesa,Burger bacon,"Pan brioche, 180 gr de carne, tomate, huevo, b...","Burger bacon Pan brioche, 180 gr de carne, tom..."
97886,Hamburguesas,Hamburguesas,Hamburguesa,Hamburguesa king size,"Doble burger, doble cheddar, cebolla colorada ...","Hamburguesa king size Doble burger, doble ched..."


#### Limpiamos caracteres

In [9]:
samples['description'] = samples['description'].apply(description_to_words)
samples['description']

33       hamburguesa vegetariana ciser hamburguesa garb...
66       godiee burger g carne lentejas pan rucula alba...
91       hamburguesa vegetariana doble hamburguesa lent...
181      hamburguesa vegetariana pan artesanal sesamo h...
258      hamburguesa vegetariana pan batta dos hamburgu...
                               ...                        
97882    jesse pinkman burger medallon g carne mayonesa...
97884    hamburguesa doble doble carne casera huevo fri...
97885    burger bacon pan brioche gr carne tomate huevo...
97886    hamburguesa king size doble burger doble chedd...
97888    hamburguesa plato dos hamburguesas cebolla hue...
Name: description, Length: 7173, dtype: object

In [10]:
sentences = list(samples['description'])

for i in range(len(sentences)):
    sentences[i] = sentences[i].split()

#### Ejemplo de producto tokenizado:

In [51]:
G = nx.MultiGraph()

In [52]:
burguer_synonyms = ['hamburguesa', 'Hamburguesa', 'burger', 'Burger', 'sanguche', 'Sanguche', 'sandwich', 'hamburguesas', 'sándwich', 'Sándwich']
for sentence in sentences:
    for (a, b) in list(itertools.combinations(sentence, 2)):
        if not (a in burguer_synonyms or b in burguer_synonyms):
            G.add_edge(a, b)

In [53]:
nx.write_gexf(G, "comiditas.gexf")

In [54]:
res = nx.isomorphism.ISMAGS(G, G)

In [60]:
asd = res.largest_common_subgraph()